In [1]:
import numpy as np
import sys
import os
import neuro
import risp
import random

from encode import Chromagram
from encode import Encoder
from data_loader import DataLoader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
} 
proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties

n_inputs = 12
n_hidden = 360
n_outputs = 12
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 1000


moa = neuro.MOA()
moa.seed(42)


In [3]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [4]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(DataLoader().chroma_files, DataLoader().numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

In [5]:
print(DataLoader().numerical_label_roots)

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 3, 

In [6]:
len(X_train)

1752

In [7]:
for i in range(10):
    print(X_temp[i])

/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/sus4_triad/D274_sus4triad_C#6_85.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/Nord_majortriad_C7_96.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_ninth/Nord_dom7ninth_G#6_92.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/sus2_triad/D274_sus2triad_C#2_37.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/sus4_triad/Nord_sus4triad_G2_43.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/ES100_majortriad_C5_72.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_ninth/D274_dom7ninth_C#4_61.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/sus4_triad/MODX8_sus4triad_E1_28.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_seventh/D274_

In [8]:
for i in range(10):
    print(y_temp[i])

4
3
11
4
10
3
4
7
6
0


In [9]:
y_train_encoded[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [10]:
loader_instance = DataLoader(folders=['major_triad', 'minor_triad'])

In [11]:
X_temp2, X_test2, y_temp2, y_test2 = train_test_split(loader_instance.chroma_files, loader_instance.numerical_label_roots, test_size=0.2, random_state=42)

In [12]:
for i in range(10):
    print(X_temp2[i])

/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/ES100_majortriad_A#5_82.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/minor_triad/ES100_minortriad_G5_79.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/MODX8_majortriad_G#5_80.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/MODX8_majortriad_E4_64.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/minor_triad/MODX8_minortriad_B1_35.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/Nord_majortriad_G5_79.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/ES100_majortriad_F4_65.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/Nord_majortriad_F#1_30.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/maj

In [13]:
for i in range(10):
    print(y_temp2[i])

1
10
11
7
2
10
8
9
1
11


In [14]:
X_temp3, X_test3, y_temp3, y_test3 = train_test_split(loader_instance.chroma_files, loader_instance.numerical_label_types, test_size=0.2, random_state=42)

In [15]:
for i in range(10):
    print(X_temp3[i])

/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/ES100_majortriad_A#5_82.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/minor_triad/ES100_minortriad_G5_79.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/MODX8_majortriad_G#5_80.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/MODX8_majortriad_E4_64.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/minor_triad/MODX8_minortriad_B1_35.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/Nord_majortriad_G5_79.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/ES100_majortriad_F4_65.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/major_triad/Nord_majortriad_F#1_30.npy
/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/maj

In [16]:
for i in range(10):
    print(y_temp3[i])

0
1
0
0
1
0
0
0
0
1


In [13]:
original_root_notes = loader_instance.get_original_labels(y_test_encoded[0].reshape(1, -1))
original_root_notes


['G#']

In [30]:
y_test_encoded[0].reshape(1, -1)

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [17]:
encoder = Encoder(X_train[2])

len(encoder.spikes)

4519

In [18]:
proc.load_network(net)

True

In [19]:
for i in range(n_outputs):
    proc.track_output_events(i)
    print(proc.track_output_events(i))

True
True
True
True
True
True
True
True
True
True
True
True


In [20]:
proc.apply_spikes(encoder.spikes)

In [21]:
proc.run(encoder.time_steps * encoder.num_bins * 12)

In [22]:
vectors = proc.output_vectors()
print(vectors)

for item in vectors:
    print(len(item))

[[0.0, 3.0, 6.0, 9.0, 13.0, 16.0, 19.0, 23.0, 25.0, 27.0, 29.0, 32.0, 34.0, 36.0, 39.0, 41.0, 43.0, 45.0, 46.0, 48.0, 51.0, 53.0, 56.0, 58.0, 61.0, 64.0, 66.0, 69.0, 73.0, 76.0, 79.0, 83.0, 86.0, 90.0, 92.0, 96.0, 100.0, 103.0, 107.0, 111.0, 114.0, 116.0, 119.0, 123.0, 126.0, 130.0, 134.0, 137.0, 139.0, 142.0, 145.0, 149.0, 152.0, 155.0, 159.0, 162.0, 166.0, 170.0, 175.0, 179.0, 183.0, 185.0, 189.0, 193.0, 197.0, 200.0, 204.0, 208.0, 212.0, 216.0, 220.0, 223.0, 227.0, 231.0, 232.0, 236.0, 240.0, 244.0, 249.0, 253.0, 255.0, 258.0, 262.0, 266.0, 270.0, 273.0, 277.0, 278.0, 282.0, 286.0, 290.0, 294.0, 298.0, 301.0, 305.0, 309.0, 314.0, 318.0, 322.0, 324.0, 328.0, 332.0, 336.0, 340.0, 344.0, 348.0, 352.0, 356.0, 361.0, 365.0, 370.0, 371.0, 375.0, 379.0, 383.0, 387.0, 391.0, 394.0, 398.0, 402.0, 406.0, 410.0, 414.0, 417.0, 422.0, 426.0, 431.0, 435.0, 440.0, 444.0, 448.0, 452.0, 456.0, 460.0, 464.0, 468.0, 472.0, 476.0, 480.0, 484.0, 487.0, 491.0, 495.0, 499.0, 503.0, 507.0, 510.0, 514.0, 51

In [23]:
vector = proc.output_vector(0)
#make this list to a numpy array

vector

[0.0,
 3.0,
 6.0,
 9.0,
 13.0,
 16.0,
 19.0,
 23.0,
 25.0,
 27.0,
 29.0,
 32.0,
 34.0,
 36.0,
 39.0,
 41.0,
 43.0,
 45.0,
 46.0,
 48.0,
 51.0,
 53.0,
 56.0,
 58.0,
 61.0,
 64.0,
 66.0,
 69.0,
 73.0,
 76.0,
 79.0,
 83.0,
 86.0,
 90.0,
 92.0,
 96.0,
 100.0,
 103.0,
 107.0,
 111.0,
 114.0,
 116.0,
 119.0,
 123.0,
 126.0,
 130.0,
 134.0,
 137.0,
 139.0,
 142.0,
 145.0,
 149.0,
 152.0,
 155.0,
 159.0,
 162.0,
 166.0,
 170.0,
 175.0,
 179.0,
 183.0,
 185.0,
 189.0,
 193.0,
 197.0,
 200.0,
 204.0,
 208.0,
 212.0,
 216.0,
 220.0,
 223.0,
 227.0,
 231.0,
 232.0,
 236.0,
 240.0,
 244.0,
 249.0,
 253.0,
 255.0,
 258.0,
 262.0,
 266.0,
 270.0,
 273.0,
 277.0,
 278.0,
 282.0,
 286.0,
 290.0,
 294.0,
 298.0,
 301.0,
 305.0,
 309.0,
 314.0,
 318.0,
 322.0,
 324.0,
 328.0,
 332.0,
 336.0,
 340.0,
 344.0,
 348.0,
 352.0,
 356.0,
 361.0,
 365.0,
 370.0,
 371.0,
 375.0,
 379.0,
 383.0,
 387.0,
 391.0,
 394.0,
 398.0,
 402.0,
 406.0,
 410.0,
 414.0,
 417.0,
 422.0,
 426.0,
 431.0,
 435.0,
 440.0,
 444.0,


In [24]:
predicted_index = proc.output_count_max(12)[0]


In [25]:
proc.output_counts()

[658, 155, 168, 428, 181, 119, 279, 225, 1775, 232, 120, 113]

In [26]:
output_count = np.array(proc.output_counts())
predicted_index = np.argmax(output_count)
print(predicted_index)

8


In [27]:
one_hot_prediction = np.zeros(n_outputs)
one_hot_prediction

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [28]:
one_hot_prediction[predicted_index]=1

In [29]:
one_hot_prediction

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [32]:
orginal_note = loader_instance.get_original_labels(one_hot_prediction.reshape(1, -1))
orginal_note

['F']

In [33]:
X_train[2]

'/home/dofo/Repos/Github/neuromorphicsnn/chord_snn/dataset/npy_chroma_chords/dom7_seventh/MODX8_dom7seventh_G#4_68.npy'

In [35]:
y_train[2]

11

In [20]:
# * 5 -> 1142

# * 6 -> 1378

# * 7 -> 1610

# * 8 -> 1676